In [27]:
import time
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fmin_l_bfgs_b
import tensorflow as tf
from keras.applications import vgg16
from keras import backend as K

In [34]:
# 图片路径
base_image_path = 'content/sy.jpg'
style_reference_image_path = 'style/style10.jpg'
result_prefix = 'output_imgs'
# 迭代次数
iterations = 5
# 图片平滑参数
total_variation_weight = 8.5e-5
# 风格及内容权值
style_weight = 1.0
content_weight = 0.025
# 原图宽高
width, height = load_img(base_image_path).size
# 结果图片高
img_nrows = 720
# 结果图片宽（等比缩放）
img_ncols = int(width * img_nrows / height)

In [29]:
# 图片预处理
def preprocess_image(image_path):
    # 使用Keras内置函数读入图片并设置为指定长宽
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    # 转为numpy array格式
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# 反向操作
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # 加上颜色均值
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [30]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

def content_loss(base, combination):
    return K.sum(K.square(combination - base))

def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, 1:, :img_ncols-1, :])
    b = K.square(x[:, :img_nrows-1, :img_ncols-1, :] - x[:, :img_nrows-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [31]:
base_image = K.variable(preprocess_image(base_image_path)) # 内容图
style_reference_image = K.variable(preprocess_image(style_reference_image_path)) # 风格图

combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

input_tensor = K.concatenate([base_image, style_reference_image, combination_image], axis=0)

In [32]:
# 载入模型
model = vgg16.VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

# 初始化loss值
loss = K.variable(0.)

layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
# 计算内容损失
loss = loss + content_weight * content_loss(base_image_features, combination_features)

# 计算风格损失
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    # 计算风格损失
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl

# 求全变差正则，计算总loss
loss = loss + total_variation_weight * total_variation_loss(combination_image)

Model loaded.


In [33]:
# loss函数关于combination_image的梯度
tf.compat.v1.disable_eager_execution()
grads = K.gradients(loss, combination_image)

outputs = [loss]
outputs += grads
f_outputs = K.function([combination_image], outputs)

In [35]:
# 获取loss和grads
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

In [36]:
# 定义方法，用于返回loss、grads
class Evaluator(object):
    def __init__(self):
        # 初始化损失值和梯度值
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        # 调用函数得到梯度值和损失值，返回损失值
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        # 返回梯度值
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [38]:
evaluator = Evaluator()

# 将内容图片作为待优化图片
x = preprocess_image(base_image_path)

# 显示原始图片
img = load_img(base_image_path, target_size=(img_nrows, img_ncols))
plt.imshow(img)
plt.axis('off')
plt.show()

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    # 使用(L-BFGS)算法来最小化loss的值
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)

    # 反向操作，加上颜色均值及'BGR'->'RGB'
    img = deprocess_image(x.copy())

    # 保存产生的新图片
    fname = result_prefix + '/_at_iteration_%d.png' % i
    print('Image saved as', fname)
    imsave(fname, img)

    # 计算迭代1次花费的时间
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

    # 显示图片
    plt.imshow(img)
    plt.axis('off')
    plt.show()